# Metabolic Pathways

## Query Pattern

Most of our information about metabolism of chemicals comes from kegg.  We are not storing kegg reactions as entities themselves.  Instead, if we have a reaction where chemical A -> chemical B, and it's catalyzed by a gene C, we create the following relationships:

```
A derives_into B
C increases_degradation_of A
C increases_synthesis_of B
```

So we can look for reactions using that pattern.

## Functions

In [1]:
import requests
import pandas as pd

robokop_server = 'robokop.renci.org'

def quick(question):
    url=f'http://{robokop_server}:80/api/simple/quick/'
    #print(url)
    response = requests.post(url,json=question)
    #print( f"Return Status: {response.status_code}" )
    if response.status_code == 200:
        return response.json()
    return {}
    #return response

In [2]:
def make_decomposition_question(chemical_identifier):
    """Given a chemical input, construct a graph query tofind chemicals that it metabolizes into, 
    and the gene that catalyzes the rxn."""
    question = {
                'machine_question': {
                    'nodes': [
                        {
                            'id': 'n0',
                            'curie': chemical_identifier,
                            'type': 'chemical_substance'
                        },
                        {
                            'id': 'n1',
                            'type': 'gene'
                        },
                        {
                            'id': 'n2',
                            'type': 'chemical_substance'
                        }
                    ],
                    'edges': [
                        {
                            'id': 'e0',
                            'source_id': 'n1',
                            'target_id': 'n0',
                            'type': 'increases_degradation_of'
                        },
                        {
                            'id': 'e1',
                            'source_id': 'n1',
                            'target_id': 'n2',
                            'type': 'increases_synthesis_of'
                        },
                        {
                            'id': 'e2',
                            'source_id': 'n0',
                            'target_id': 'n2',
                            'type': 'derives_into'
                        }
                    ]
                }
            }
    return question

In [3]:
import pandas as pd
def answers2frame(graph_answers, input_chem):
    """Convert the answer from a decomposition query into a pandas DataFrame"""
    answers = []
    for graph_answer in graph_answers['answers']:
        try:
            for node in graph_answer['nodes']:
                if node['type'] == 'gene':
                    gene = node
                elif node['type'] == 'chemical_substance':
                    if node['id'] == input_chem:
                        reactant=node
                    else:
                        product=node
            ans = { 'score': graph_answer['score'],
                    'reactant': reactant['name'] if 'name' in reactant else reactant['id'],
                    'gene': gene['name'],
                    'product': product['name'] if 'name' in product else product['id'],
                    'product_id': product['id']
                  }
        except IndexError:
            #Can this happen here?
            continue
        answers.append(ans)
    df = pd.DataFrame(answers)
    ordered_columns = ['score','reactant','gene','product','product_id']
    df = df[ordered_columns]
    return df

## Example

Suppose we wanted to investigate the metabolism of Prostaglandin H2 (CHEBI:15554).  We can do that with the following code, which creates the question, calls the ROBOKOP quick service, then creates and displays a data frame of the answers.

In [9]:
chem_id = 'CHEBI:15554'  #Prostaglandin H2
q = make_decomposition_question(chem_id)
a = quick(q)

http://robokop.renci.org:80/api/simple/quick/


In [11]:
df = answers2frame(a,chem_id)
df

,score,reactant,gene,product,product_id
0,0,prostaglandin H2,HPGDS,prostaglandin D2,CHEBI:15555
1,0,prostaglandin H2,TBXAS1,thromboxane A2,CHEBI:15627
2,0,prostaglandin H2,PTGES,prostaglandin E2,CHEBI:15551
3,0,prostaglandin H2,PTGES2,prostaglandin E2,CHEBI:15551
4,0,prostaglandin H2,PTGDS,prostaglandin D2,CHEBI:15555
5,0,prostaglandin H2,PTGES3,prostaglandin E2,CHEBI:15551
6,0,prostaglandin H2,PRXL2B,prostaglandin F2alpha,CHEBI:15553


## Tracing Pathways Recursively

The code above asks about a single step.  Given a chemical, what does it metabolize into?  Now, we can write a function to recursively take the outputs and pass them back as inputs, so that we can trace out multiple steps.  The tricky thing here is that there are a bunch of 'balancing' chemicals that we don't want to consider part of our pathways.  These are things like water or carbon dioxide that will often appear as the product of a reaction. But we don't want to then use them to look for reactions where they are on the left hand side, because we will get an explosion of reactions.

In [15]:
def find_pathway(input_chemical_with_label,accumulator,max_products=0):
    """Pass in the identifier for the chemical you want the metabolic pathway of, and an empty dict for accumulator.
    Results will be passed out in accumulator."""
    dead_ends = ['CHEBI:16526', #carbon dioxide
                 'CHEBI:15741', #succinic acid / succinate
                 'CHEBI:15846', #NAD(+)
                 'CHEBI:18009', #NADP(+)
                 'CHEBI:44409', #NADP Zwitterion
                 'CHEBI:58349', #NADP(3-)
                 'CHEBI:16908', #NADH
                 'CHEBI:16474', #NADPH
                 'CHEBI:15377', #Water
                 'CHEBI:15379', #dioxygen
                 'CHEBI:16526', #carbon dioxide
                 'CHEBI:16842', #formaldehyde
                 'CHEBI:17659', #UDP
                 'CHEBI:15713', #ADP
                 'CHEBI:16761', #ADP? 
                 'CHEBI:16134', #ammonia
                 'CHEBI:16240', #hydrogen peroxide
                 'CHEBI:24636', #proton
                 'CHEBI:15378', #hydron 
                 'CHEBI:29888', #diphosphoric acid
                 'CHEBI:30769', #citric acid
                ]
    input_chemical = input_chemical_with_label[0]
    input_label = input_chemical_with_label[1]
    if input_chemical_with_label not in accumulator and input_chemical not in dead_ends:
        print(input_chemical, input_label)
        q = make_decomposition_question(input_chemical)
        a = quick(q)
        answers = []
        if 'answers' in a:
            kg_nodes_by_id = { n['id']: n for n in a['knowledge_graph']['nodes'] }
            for answer in a['answers']:
                gene_id = answer['node_bindings']['n1']
                gene = kg_nodes_by_id[gene_id]
                product_id = answer['node_bindings']['n2']
                product = kg_nodes_by_id[product_id]
                if product['id'] not in dead_ends:
                    answers.append( {'gene':gene, 'product':product, 'score':answer['score']})
            accumulator[input_chemical_with_label] = answers
            distinct_products = set([(answer['product']['id'],answer['product']['name']) for answer in answers])
            #print('  ',len(distinct_products))
            if max_products > 0 and len(distinct_products) <= max_products:
                for product in distinct_products:
                    find_pathway(product,accumulator)
            

In [14]:
q = make_decomposition_question('CHEBI:50616')
a = quick(q)
import json
print(json.dumps(a,indent=4))

"No results found"


In [11]:
kg_nodes_by_id = { n['id']: n for n in a['knowledge_graph']['nodes'] }

In [16]:
pathway = {}
chem_id = ('CHEBI:17053', 'L-aspartic acid')
#chem_id = ('CHEBI:6121','ketamine') #nothing
#chem_id = ('CHEBI:16236', 'ethanol')
#chem_id = ('CHEBI:16714' , 'codeine')
#chem_id = ('CHEBI:15554', 'Prostaglandin H2')
#chem_id = ('CHEBI:18243', 'dopamine')
#chem_id = ('CHEBI:16436','CDP-choline')
#chem_id = ('CHEBI:27732','caffeine')
#chem_id = ('CHEBI:41774', 'tamoxifen')
#chem_id = ('CHEBI:27432', 'alpha-linolenic acid')

find_pathway(chem_id,pathway,max_products=20)

CHEBI:17053 L-aspartic acid
CHEBI:26078 phosphoric acid
CHEBI:50616 iminoaspartic acid
CHEBI:17196 L-asparagine
CHEBI:21547 N-acetyl-L-aspartic acid
CHEBI:57966 beta-alanine zwitterion
CHEBI:16015 L-glutamic acid
CHEBI:58048 L-asparagine zwitterion
CHEBI:30744 oxaloacetic acid
CHEBI:18319 SAICAR
CHEBI:16958 beta-alanine
CHEBI:17552 GDP
CHEBI:16027 adenosine 5'-monophosphate
CHEBI:15859 N-carbamoyl-L-aspartic acid
CHEBI:18237 glutamic acid


In [17]:
flatter = []
for k,v in pathway.items():
    for o in v:
        flatter.append({'reactant':k[0],
                        'reactantname': k[1],
                        'geneid':o['gene']['id'], 
                        'genename':o['gene']['name'] if 'name' in o['gene'] else '', 
                        'productid': o['product']['id'], 
                        'productname': o['product']['name'] if 'name' in o['product'] else '',
                        'score': o['score']})

In [19]:
df = pd.DataFrame(flatter)
ordered_columns = ['reactant','reactantname','genename','productname','productid']
df = df[ordered_columns]

df.sort_values(by=['reactantname','productname','genename'],inplace=True)

from IPython.display import display
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

,reactant,reactantname,genename,productname,productid
144,CHEBI:17552,GDP,AK9,GTP,CHEBI:15996
145,CHEBI:17552,GDP,NME1,GTP,CHEBI:15996
134,CHEBI:17552,GDP,NME1-NME2,GTP,CHEBI:15996
137,CHEBI:17552,GDP,NME2,GTP,CHEBI:15996
133,CHEBI:17552,GDP,NME3,GTP,CHEBI:15996
139,CHEBI:17552,GDP,NME4,GTP,CHEBI:15996
141,CHEBI:17552,GDP,NME6,GTP,CHEBI:15996
138,CHEBI:17552,GDP,NME7,GTP,CHEBI:15996
142,CHEBI:17552,GDP,ENTPD1,guanosine 5'-monophosphate,CHEBI:17345
136,CHEBI:17552,GDP,ENTPD3,guanosine 5'-monophosphate,CHEBI:17345


In [77]:
products_and_reactants = df[ ['reactant','reactantname','productid','productname']]
products_and_reactants.drop_duplicates(inplace=True)
products_and_reactants

/home/bizon/anaconda2/envs/robokop/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,reactant,reactantname,productid,productname
0,CHEBI:16714,codeine,CHEBI:17303,morphine
24,CHEBI:16714,codeine,CHEBI:80579,norcodeine
25,CHEBI:16714,codeine,CHEBI:80580,Codeine-6-glucuronide
44,CHEBI:17303,morphine,CHEBI:80581,Morphine-6-glucuronide
45,CHEBI:17303,morphine,CHEBI:80631,Morphine-3-glucuronide
69,CHEBI:17303,morphine,CHEBI:7633,Normorphine
